Text Generation with Markov Chains in Python
============================================

In [62]:
import tensorflow as tf

In [114]:
files = ['text/grimm_tales.txt', 'text/little_red_riding_hood.txt',\
         'text/robin_hood_prologue.txt']
# files = ['text/grimm_tales.txt']
text = ''
for f in files:
    with open(f, 'r') as f:
        text += f.read()


In [115]:
print(text[:150])

THE GOLDEN BIRD

A certain king had a beautiful garden, and in the garden stood a tree
which bore golden apples. These apples were always counted, and


Weather as a Markov Chain
-------------------------

![alt text](images/markov_weather.png "Weather")

Matrix representation (rows are current state, columns are next state):

| | Sunny | Cloudy | Rainy |
| --- | --- | --- | --- |
| **Sunny** | 0.6 | 0.1 | 0.3 |
| **Cloudy** | 0.3 | 0.3 | 0.4 |
| **Rainy** | 0.3 | 0.2 | 0.5 |


Text as a Markov Chain
----------------------

**The cat ran over the dog.**

![alt text](images/markov_text1.png "Text")

Matrix representation (rows are current state, columns are next state):

| | the | cat | ran | over | dog | . |
| --- | --- | --- | --- | --- | --- | --- |
| **the** | 0 | 0.5 | 0 | 0 | 0.5 | 0 |
| **cat** | 0 | 0 | 1 | 0 | 0 | 0 |
| **ran** | 0 | 0 | 0 | 1 | 0 | 0 |
| **over** | 1 | 0 | 0 | 0 | 0 | 0 |
| **dog** | 0 | 0 | 0 | 0 | 0 | 1 |
| **.** | 0 | 0 | 0 | 0 | 0 | 1 |



### Define states as the distinct word tokens

In [116]:
import re

text = re.sub("[^A-z,.!?'\n ]+", "", text) #cleaning characters to keep only the letters and punctuation. Important
                                           #to keep space and \n as well.  
text = re.sub("([.,!?])", r" \1 ", text) # we want to keep punctuations as token, so we replace with two spaces around. 
    #Here we are using a capture syntax https://www.lzone.de/examples/Python%20re.sub  
    #need to put Parentheses around the squared brackets and then the '\1'.
    
    #Needed to add the r".." to indicate raw string to read the \ correctly
    
tokens = text.lower().split()
distinct_states = list(set(tokens))
print(distinct_states[:100])

['prince', 'assured', 'face', 'butcher', 'missing', 'brilliant', 'nocked', 'rows', 'hanged', 'branches', 'light', 'restingplace', 'whatsoever', 'conversation', 'swam', 'mouths', 'mouses', 'suspecting', 'boar', 'sausage', 'jingling', 'painted', 'flown', 'west', 'woes', 'envy', 'marks', 'entire', 'divided', 'walk', 'suck', 'unconsumed', 'cask', 'intention', 'nibbled', 'sounding', 'stuck', 'plaything', 'sin', 'month', 'dismay', 'talking', 'wetted', 'neigh', 'impatience', 'dismal', 'soldier', 'scarce', 'dangling', 'heard', 'belaboured', 'meddling', 'peacefully', 'help', 'relations', 'furs', 'nostrils', 'frolickd', 'advice', 'rolls', 'danger', 'roved', 'park', 'grease', 'impetuous', 'cur', 'crash', 'swim', 'handed', 'unbound', 'collapsed', 'height', 'seized', 'fun', 'push', 'scissorgrinder', 'willow', 'sinner', 'consequence', 'disenchanted', 'smiling', 'punishment', 'pot', 'enemys', 'snuff', 'gorged', 'happens', 'ranged', 'certainly', 'returning', 'dumpling', 'woefully', 'known', 'combed', 

### Define transition matrix

In [117]:
from scipy.sparse import csr_matrix
m = csr_matrix(
    (len(distinct_states), len(distinct_states)), #Number of rows, number of columns. 
    dtype = int                                   #Defining the type of the elements. 
        )

state_index = dict([(state, idx_num) for idx_num, state in enumerate(distinct_states)])
    #We want numbers to indicate rows and column, so we make a dictionary. List can't be a dict key, so we use tuples

### Count transitions and fill in transition matrix

In [118]:
%%time
for i in range(len(tokens)-1):
    row = state_index[tokens[i]]
    col = state_index[tokens[i+1]]
    m[row, col]+=1
#     m._set_intXint(row, col, )

CPU times: user 29.3 s, sys: 3.91 ms, total: 29.3 s
Wall time: 29.3 s


### Generate new text

In [123]:
%%time
import numpy as np
import scipy

# We define randomly the first word we want to use. 
start_state_index  = np.random.randint(len(distinct_states))
state = distinct_states[start_state_index]

#We use num_sentences to keep track of how many sentences we want. 
num_sentences = 0

#We capitalize the first word. 
output = state.capitalize()

#We use this variable to remember when we need to capitalize or not. 
capitalize = False

while num_sentences < 3:
    #We start by taking all the potential following words for the first state we chose. 
    row = m[state_index[state], :]
    probabilities = (row / row.sum()).toarray()[0] #normalizing the values
#     probabilities =  tf.nn.softmax([float(x) for x in row.toarray()[0]])
    
#     if np.sum(probabilities) !=1.0:
#         print('normalize', np.sum((row / row.sum()).toarray()[0]))
#         print('tf softmax', np.sum(tf.nn.softmax([float(x) for x in row.toarray()[0]])))
#         print('sc softmax', np.sum(scipy.special.softmax(row.toarray()[0])))
        
    next_state_index = np.random.choice( #allow to sample following a distrib.
        len(distinct_states),
        1,
        p = probabilities #indicate the distrubution of each integer. 
    )
    
    next_state = distinct_states[next_state_index[0]]
    
    if next_state in ('.', '!', '?'):
        output += next_state + '\n\n'
        capitalize = True
        num_sentences += 1
        
    elif next_state == ',':
        output += next_state #no space
        
    else:
        
        if capitalize:
            output += next_state.capitalize()
            capitalize = False
        else:
            output += ' ' + next_state

    
    state = next_state
print(output)

AttributeError: 'tuple' object has no attribute 'capitalize'

k-Word Markov Chain
-------------------

**The cat ran over the dog.**

![alt text](images/markov_text2.png "Text")

Matrix representation (rows are current state, columns are next state):

| | the cat | cat ran | ran over | over the | the dog | dog. |
| --- | --- | --- | --- | --- | --- | --- |
| **the cat**  | 0 | 1 | 0 | 0 | 0 | 0 |
| **cat ran**  | 0 | 0 | 1 | 0 | 0 | 0 |
| **ran over** | 0 | 0 | 0 | 1 | 0 | 0 |
| **over the** | 0 | 0 | 0 | 0 | 1 | 0 |
| **the dog**  | 0 | 0 | 0 | 0 | 0 | 1 |
| **dog.**     | 0 | 0 | 0 | 0 | 0 | 1 |



### Define states as consecutive token pairs

In [120]:
k = 3
tokens = text.lower().split()
states = [ tuple(tokens[i:i+k]) for i in range(len(tokens) - k+1)] 
    #Need tuples because list can be a key in a dict.
    
distinct_states = list(set(states))

### Define and fill transition matrix

In [121]:
%%time
from scipy.sparse import csr_matrix

m = csr_matrix(
    (len(distinct_states), len(distinct_states)), 
    dtype = int)

state_index = dict( 
    [(state, idx_num) for idx_num, state in \
                   enumerate(distinct_states)])

for i in range(len(tokens)-k):
    state = tuple(tokens[i:i+k])
    next_state = tuple(tokens[i+1:i+k+1])
    row = state_index[state]
    col = state_index[next_state]
    m[row, col]+=1
#     m._set_intXint(row, col, )

CPU times: user 2min 17s, sys: 11.7 ms, total: 2min 17s
Wall time: 2min 17s


### Generate new text

In [122]:
%%time
import numpy as np

start_state_index  = np.random.randint(len(distinct_states))
state = distinct_states[start_state_index]

num_sentences = 0
output = ' '.join(state).capitalize()
capitalize = False

while num_sentences < 3:
    
    row = m[state_index[state], :]
    probabilities = row / row.sum() #normalizing the values
    probabilities = probabilities.toarray()[0] #[0] to get only the values. 
    
    next_state_index = np.random.choice( #allow to sample following a distrib.
        len(distinct_states),
        1,
        p = probabilities #indicate the distrubution of each integer. 
    )
    
    next_state = distinct_states[next_state_index[0]]
    
    if next_state[-1] in ('.', '!', '?'):
#         print('punctuation ')
        output += next_state[-1] + '\n\n'
#         print(output)

        capitalize = True
        num_sentences += 1
        
    elif next_state[-1] == ',':
#         print( ' , - ', next_state) 
        output += next_state[-1] #no space
#         print(output)
        
    else:
        
        if capitalize:
#             print('capita')
            output += next_state[-1].capitalize()
#             print(output)

            capitalize = False
        else:
#             print('notmal')
            output += ' ' + next_state[-1]
#             print(output)

    
    state = next_state
print(output)

, but also because he had poached upon the king's venison, washed down with draughts of ale of october brewing.

Not only robin himself but all the gifts of the first giant.

That is a very odd and uncommon name, is it a usual one in your family?


CPU times: user 52.6 ms, sys: 1 µs, total: 52.6 ms
Wall time: 51.5 ms
